In [177]:
import os
import xlrd
import xlwt
from xlutils.copy import copy
import pandas as pd
import numpy as np
from openpyxl import load_workbook

os.chdir("/Users/mauri/Documents/Python Scripts/RFP Import Scripts")
os.listdir('.')

['.ipynb_checkpoints',
 'Concord_Script_1stPart.ipynb',
 'Concord_Script_RFP-New Version 2.0.ipynb',
 'FPC- New Data with New Forecast.xlsx',
 'FPC.ipynb',
 'TEST_MAPPING - Copy.xlsx',
 'TEST_MAPPING.xls',
 'TEST_MAPPING.xlsx']

In [178]:
file = pd.ExcelFile('FPC- New Data with New Forecast.xlsx')
sheet_names = file.sheet_names
#print(sheet_names)   -- Take out the hashtag to see the model names and scenarios.

In [179]:
def extract(info):
    """Function to extract desire information"""
    res = []
    for sheet in sheet_names:
        data = pd.read_excel(file, sheet)
        for index, row in data.iterrows():
            for count in range(0, row.size):
                if row[count] == info:
                    res.append([sheet,row[count],row[count+1]])
    return res

#Selection of Equipment and Scenario 
def select(dtf,Equipment,Scenario):
    """F(x) to query data from specific machine and scenario"""
    res = []
    for index, row in dtf.iterrows():
            for count in range(0, row.size):
                if row[count] == Equipment+Scenario:
                    res.append([row[count+2]])
    return np.asarray(res)

def importxls(dtf):
    """F(x) creates dtf to be imported to xls"""
    new = dtf[0].str.split("(", n = 1, expand = True).replace('\)','',regex=True)
    # making seperate first name column from new data frame 
    dtf["Model"]= new[0] 

    # making seperate last name column from new data frame 
    dtf["Scenario"]= new[1] 

    # Dropping old Name columns 
    dtf.drop(columns =[0], inplace = True) 
    columns = dtf.columns.tolist()
    columns = columns[-2:] + columns[:-2]
    dtf = dtf[columns]
    dtf.rename(columns={1: "Line Item", 2: "Amount"}, inplace=True)
    return dtf

#Extract Total Cycle Time 
res_TCT = pd.DataFrame(extract('Total Cycle Time:'))

#Extract Total Fuel Usage 
res_TFU = pd.DataFrame(extract('Total Fuel Usage:'))

#Extract Payload
res_PLD = pd.DataFrame(extract('Payload:')).groupby([0,1]).mean().reset_index()

In [180]:
# # Select Model & Scenario. Just Run the code afterwards without modifying anything. 
# Model = '793F 2270hp SR'
# Scenario = '(Haul 1)'

# CycleTime = select(res_TCT,Model,Scenario)
# print(CycleTime)

# FuelUsage = select(res_TFU,Model,Scenario )
# print(FuelUsage)

# Payload = select(res_PLD,Model,Scenario )
# print(Payload)

# rb = xlrd.open_workbook('TEST_MAPPING - Copy.xlsx')  
# wb = copy(rb)
# sheet = wb.get_sheet('data_entry')
# sheet.write(344, 2, 'BITCH')
# wb.save('TEST_MAPPING - Copy.xlsx')

[[52.39]]
[[284.41]]
[[240.39]]


In [181]:
CycleTimes = importxls(res_TCT)
FuelUsage = importxls(res_TFU)
Payload = importxls(res_PLD)
FPC = pd.concat([CycleTimes,FuelUsage,Payload])

In [174]:
book = load_workbook('TEST_MAPPING.xlsx')
writer = pd.ExcelWriter('TEST_MAPPING.xlsx', engine = 'openpyxl')
writer.book = book

FPC.to_excel(writer, sheet_name = 'FCP',index=False, index_label= None)

writer.save()
writer.close()